In [1]:
from functools import wraps

In [2]:
# Decorator to prime coroutine
def coroutine(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        cr = func(*args, **kwargs)
        next(cr) # No need to prime the coroutine anymore, the decorator does it for you
        return cr
    return wrapper

In [3]:
def example_grep(pattern):
    print('Looking for {}'.format(pattern))
    try:
        while True:
            line = yield
            if pattern in line:
                print(line)
    except GeneratorExit:
        print('Closing. No more grep for you !')

In [4]:
g = example_grep('python')
next(g) # Or just use the decorator above for auto priming

Looking for python


In [5]:
g.send('Hello World')
g.send('I am a pythonista.')
g.send('c++ or python')
g.send('never mind')

I am a pythonista.
c++ or python


In [6]:
g.close() # Stop the generator. Will throw an exception that you need to catch

Closing. No more grep for you !


In [7]:
@coroutine
def printer():
    while True:
        line = yield
        print(line)

In [8]:
@coroutine
def broadcast(targets):
    while True:
        line = yield
        for target in targets:
            target.send(line)

In [9]:
@coroutine
def grep(pattern, publisher):
    while True:
        line = yield
        if pattern in line:
            publisher.send(line+' ({})'.format(pattern))

In [10]:
p = printer()
x = grep('python', p)
y = grep('ml', p)
b = broadcast([x, y])

In [11]:
b.send('Hello')
b.send('All hail the BDFL')
b.send('python is cool')
b.send('ml is in demand')
b.send('C++ or Fortran')
b.send('ml uses python') # This will be repeated

python is cool (python)
ml is in demand (ml)
ml uses python (python)
ml uses python (ml)


In [12]:
import xml.sax

In [13]:
class EventHandler(xml.sax.ContentHandler):
    def __init__(self, target):
        self.target = target
    def startElement(self,name,attrs):
        self.target.send(('start',(name,attrs._attrs)))
    def characters(self,text):
        self.target.send(('text',text))
    def endElement(self,name):
        self.target.send(('end',name))

# Output is too long - don't run 
# xml.sax.parse('coroutines/allroutes.xml', EventHandler(printer())) 

## OS

In [14]:
class Task:
    taskid = 0
    def __init__(self, target):
        Task.taskid += 1
        self.tid = Task.taskid # Task id
        self.target = target   # Target coroutine
        self.sendval = None    # Value to send to coroutine
        
    def run(self):
        return self.target.send(self.sendval)

In [15]:
# Example
def foo():
    print('foo - Part 1')
    yield
    print('foo -Part 2')
    yield
    
t1 = Task(foo())
print('Starting foo()')
t1.run()
print('Resuming foo()')
t1.run()
try:
    t1.run()
except StopIteration:
    print('Task finished')

Starting foo()
foo - Part 1
Resuming foo()
foo -Part 2
Task finished


In [16]:
from queue import Queue

In [17]:
class Scheduler:
    def __init__(self):
        self.ready = Queue() # Queue of tasks ready to run
        self.taskmap = {}
        
    def new(self, target):
        newtask = Task(target)
        self.taskmap[newtask.tid] = newtask # Keeps track of all active tasks using their taskid
        self.schedule(newtask) 
        return newtask.tid
        
    # Puts something to the ready queue
    def schedule(self, task):
        self.ready.put(task)
        
    def mainloop(self):
        while self.taskmap:
            task = self.ready.get()
            try:
                result = task.run()  # Only runs to the next yield 
                self.schedule(task)
            except StopIteration:
                del self.taskmap[task.tid] # The task has finished running. Remove it from taskmap

In [18]:
def bar():
    print("I'm bar")
    yield
    print("I am friends with foo.")
    yield

In [19]:
sched = Scheduler()
sched.new(foo())
sched.new(bar())
sched.mainloop()

foo - Part 1
I'm bar
foo -Part 2
I am friends with foo.
